In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve, f1_score
df = pd.read_excel('DataSet_w_NA (1).xlsx', sheet_name="Испорченные факты").dropna()
df

,Номер заказа,Возврат,Дата заказа,Дата отправки,Режим отправки,Факты.Контрагент ID,Факты.Товар ID,"Продажи, руб","Продажи, шт",Скидка,Маржинальная прибыль,Повторение заказа,Повторение контрагента,Повторение товара
0,2014-100279,Нет,2015-09-10 00:00:00,2015-09-14 00:00:00,Стандарт,CLNT0000422,PROD0000870,11190.0,2.0,0.0,5371.20,1,7,3
1,2014-100293,Нет,2015-09-14 00:00:00,2015-09-18 00:00:00,Стандарт,CLNT0000784,PROD0000646,45528.0,6.0,0.2,15934.80,1,4,8
2,2014-100328,Нет,2015-07-28 00:00:00,2015-08-03 00:00:00,Стандарт,CLNT0000342,PROD0000187,1964.0,1.0,0.2,662.85,1,4,12
3,2014-100363,Нет,2015-10-08 00:00:00,2015-10-15 00:00:00,Стандарт,CLNT0000146,PROD0001095,1184.0,2.0,0.2,414.40,2,12,7
4,2014-100363,Нет,2015-10-08 00:00:00,2015-10-15 00:00:00,Стандарт,CLNT0000146,PROD0001616,9504.0,3.0,0.2,3445.20,2,12,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10288,2018-169929,Нет,2019-03-25 00:00:00,2019-03-28 00:00:00,Первый класс,CLNT0000182,PROD0000869,19950.0,5.0,0.0,5187.00,1,14,6
10289,2018-169978,Нет,2019-06-24 00:00:00,2019-06-28 00:00:00,Стандарт,CLNT0000575,PROD0000293,135882.0,2.0,0.1,24156.80,2,11,11
10290,2018-169978,Нет,2019-06-24 00:00:00,2019-06-28 00:00:00,Стандарт,CLNT0000575,PROD0001131,7188.0,3.0,0.2,2425.95,2,11,4
10291,2018-169999,Нет,2018-12-19 00:00:00,2018-12-26 00:00:00,Стандарт,CLNT0000013,PROD0000549,5784.0,3.0,0.2,1879.80,2,11,13


In [9]:
table = pd.pivot_table(df, index='Факты.Товар ID', values = ['Продажи, шт', 'Продажи, руб', 'Маржинальная прибыль',
                                                             'Повторение заказа', 'Повторение товара'],
                       aggfunc={'Продажи, шт': [np.median, np.sum],
                              'Продажи, руб': [np.median, np.sum],
                              'Повторение заказа': np.sum,
                              'Маржинальная прибыль': np.sum})
table.columns = table.columns.map('_'.join)
table.reset_index()
table

,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_median","Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum"
Факты.Товар ID,,,,,,
PROD0000001,-28160.70,6,144078.0,631978.5,2.5,12.0
PROD0000002,963721.00,32,396467.5,5318764.0,3.5,51.0
PROD0000003,32200.55,18,7310.0,76755.0,2.0,23.0
PROD0000004,-266380.65,17,504904.5,3621384.0,4.0,29.0
PROD0000005,31874.40,18,27960.0,150984.0,5.0,24.0
...,...,...,...,...,...,...
PROD0001890,18223.65,3,161988.0,161988.0,3.0,3.0
PROD0001891,-82476.90,6,112468.5,112468.5,3.0,3.0
PROD0001892,27680.80,1,98860.0,98860.0,4.0,4.0


In [10]:
total_sale=table['Продажи, руб_sum'].sum()
table['Доля']=table['Продажи, руб_sum']/total_sale * 100
table

,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_median","Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля
Факты.Товар ID,,,,,,,
PROD0000001,-28160.70,6,144078.0,631978.5,2.5,12.0,0.055028
PROD0000002,963721.00,32,396467.5,5318764.0,3.5,51.0,0.463122
PROD0000003,32200.55,18,7310.0,76755.0,2.0,23.0,0.006683
PROD0000004,-266380.65,17,504904.5,3621384.0,4.0,29.0,0.315325
PROD0000005,31874.40,18,27960.0,150984.0,5.0,24.0,0.013147
...,...,...,...,...,...,...,...
PROD0001890,18223.65,3,161988.0,161988.0,3.0,3.0,0.014105
PROD0001891,-82476.90,6,112468.5,112468.5,3.0,3.0,0.009793
PROD0001892,27680.80,1,98860.0,98860.0,4.0,4.0,0.008608


In [11]:
table = table.sort_values(by=('Продажи, руб_sum'), ascending=False)
table

,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_median","Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля
Факты.Товар ID,,,,,,,
PROD0001406,1.259996e+07,17,5599984.0,30799912.0,4.0,20.0,2.681845
PROD0000454,3.876520e+06,23,1143891.0,13726692.0,3.0,31.0,1.195226
PROD0001430,-9.055392e+05,7,11319240.0,11319240.0,6.0,6.0,0.985602
PROD0000488,1.746230e-10,15,1401960.0,10935288.0,5.0,39.0,0.952170
PROD0001187,1.116753e+06,33,896990.0,9911739.5,3.0,37.0,0.863046
...,...,...,...,...,...,...,...
PROD0001871,1.578500e+03,1,3850.0,3850.0,2.0,2.0,0.000335
PROD0001887,1.193400e+03,1,3536.0,3536.0,2.0,2.0,0.000308
PROD0001779,1.555200e+03,3,3240.0,3240.0,1.0,1.0,0.000282


In [12]:
table=table.assign(sum_d=table['Доля'].cumsum())
table

,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_median","Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля,sum_d
Факты.Товар ID,,,,,,,,
PROD0001406,1.259996e+07,17,5599984.0,30799912.0,4.0,20.0,2.681845,2.681845
PROD0000454,3.876520e+06,23,1143891.0,13726692.0,3.0,31.0,1.195226,3.877071
PROD0001430,-9.055392e+05,7,11319240.0,11319240.0,6.0,6.0,0.985602,4.862673
PROD0000488,1.746230e-10,15,1401960.0,10935288.0,5.0,39.0,0.952170,5.814843
PROD0001187,1.116753e+06,33,896990.0,9911739.5,3.0,37.0,0.863046,6.677889
...,...,...,...,...,...,...,...,...
PROD0001871,1.578500e+03,1,3850.0,3850.0,2.0,2.0,0.000335,99.999089
PROD0001887,1.193400e+03,1,3536.0,3536.0,2.0,2.0,0.000308,99.999396
PROD0001779,1.555200e+03,3,3240.0,3240.0,1.0,1.0,0.000282,99.999679


In [13]:
table.loc[(table['sum_d'] <= 80), 'ABC'] = 'A'
table.loc[(table['sum_d'] > 80) & (table['sum_d'] <= 95), 'ABC'] = 'B'
table.loc[(table['sum_d'] > 95), 'ABC'] = 'C'
table

,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_median","Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля,sum_d,ABC
Факты.Товар ID,,,,,,,,,
PROD0001406,1.259996e+07,17,5599984.0,30799912.0,4.0,20.0,2.681845,2.681845,A
PROD0000454,3.876520e+06,23,1143891.0,13726692.0,3.0,31.0,1.195226,3.877071,A
PROD0001430,-9.055392e+05,7,11319240.0,11319240.0,6.0,6.0,0.985602,4.862673,A
PROD0000488,1.746230e-10,15,1401960.0,10935288.0,5.0,39.0,0.952170,5.814843,A
PROD0001187,1.116753e+06,33,896990.0,9911739.5,3.0,37.0,0.863046,6.677889,A
...,...,...,...,...,...,...,...,...,...
PROD0001871,1.578500e+03,1,3850.0,3850.0,2.0,2.0,0.000335,99.999089,C
PROD0001887,1.193400e+03,1,3536.0,3536.0,2.0,2.0,0.000308,99.999396,C
PROD0001779,1.555200e+03,3,3240.0,3240.0,1.0,1.0,0.000282,99.999679,C
